# Staż Techmo - zadania

Poniżej znajdują się 3 zadania, które należy rozwiązać poprzez napisanie odpowiedniego kodu (w komórkach pod treścią zadania w pliku **staz.ipynb**), pozwalającego na wykonanie wymaganych w zadaniu obliczeń i prezentacje wyników. Przy rozwiązywaniu zadań można korzystać z dowolnych paczek dostępnych w managerze Pip np. numpy, pandas, librosa itp.

Notebook z rozwiązaniem oraz dane wrzuć na swojego githuba razem z plikiem *requirements.txt* pozwalającym na łatwą instalacje wszystkich wymaganych paczek.

## 1. Przetwarzanie sygnałów

* Wczytaj plik dźwiękowy *audio.wav*
* Wstaw widget z audioplayerem tak, aby można było odsłuchać plik
* Narysuj **oscylogram**
* Wyznacz i narysuj przebieg tonu podstawowego **F0**
* Wyznacz i narysuj w skali decybelowej krótkoczasowe widmo sygnału (**STFT**). Dobierz odpowiednie wg Ciebie parametry analizy. Uzasadnij swój wybór.
* Wyznacz i narysuj w skali decybelowej wartości **MFCC** (13 współczynników). Parametry analizy pozostaw takie jak w kroku poprzednim.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa, librosa.display
import IPython.display as ipd

sound, fs = librosa.load("audio.wav")
ipd.Audio(sound,rate= fs)

#oscillogram
t = len(sound)/fs

time = np.arange(0, t, 1/fs)
plt.plot(time, sound)
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")
plt.show()

#F0
f0, voiced_flag, voiced_probs = librosa.pyin(sound, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
times = librosa.times_like(f0)
D = librosa.amplitude_to_db(np.abs(librosa.stft(sound)), ref=np.max)
fig, ax = plt.subplots()
img = librosa.display.specshow(D, x_axis='time', y_axis='log', ax=ax)
ax.set(title='Fundamental frequency estimation', ylabel= "Frequency [Hz}", xlabel= "Time [s]")
fig.colorbar(img, ax=ax, format="%+2.f dB")
ax.plot(times, f0, label='f0', color='cyan', linewidth=3)
ax.legend(loc='upper right')
plt.show()

#STFT
hop_length = 256 # 50% overlap to avoid spectral leakage
n_fft = 512 # analysis window of approx. 20 ms len to obtain a good resolution in the frequency domain and time domain
X = librosa.stft(sound, n_fft=n_fft, hop_length=hop_length)
S = librosa.amplitude_to_db(abs(X))

plt.figure(figsize=(15, 5))
librosa.display.specshow(S, sr=fs, hop_length=hop_length, x_axis='time', y_axis='linear')
plt.colorbar(format='%+2.0f dB')
plt.ylabel("Frequency [Hz]")
plt.xlabel("Time [s]")
plt.show()

#MFCC
MFCC = librosa.feature.mfcc(sound, fs, n_mfcc=13)
logS = librosa.power_to_db(abs(MFCC))
plt.figure(figsize=(15, 5))
librosa.display.specshow(logS, sr=fs, hop_length=hop_length, x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.ylabel("MFCC Coefficient")
plt.xlabel("Time [s]")
plt.show()

## 2. Metryki

W pliku *predicted.json* zapisane są wyniki rozpoznania systemu ASR, służącego do rozpoznawania cyfr.

Dla każdej cyfry od 0 do 9 w słowniku przedstawione są wartości, jakie system rozpoznał dla kolejnych nagrań danej cyfry. 

Oblicz i przedstaw w formie tabeli **macierz pomyłek** (confusion matrix) oraz oblicz **skuteczność** systemu.

In [ ]:
import json
import numpy as np
from prettytable import PrettyTable

pred = np. zeros((10,10))
s = 0
TP = 0

with open('predicted.json') as json_file:
    data = json.load(json_file)

for i in data:
    s = s + (len(data[i]))
    for k in data[i]:
        if k == '0':
            pred[int(i), 0] = pred[int(i),0] + 1
        elif k == '1':
            pred[int(i), 1] = pred[int(i),1] + 1
        elif k == '2':
            pred[int(i), 2] = pred[int(i), 2] + 1
        elif k == '3':
            pred[int(i), 3] = pred[int(i), 3] + 1
        elif k == '4':
            pred[int(i), 4] = pred[int(i), 4] + 1
        elif k == '5':
            pred[int(i), 5] = pred[int(i), 5] + 1
        elif k == '6':
            pred[int(i), 6] = pred[int(i), 6] + 1
        elif k == '7':
            pred[int(i), 7] = pred[int(i), 7] + 1
        elif k == '8':
            pred[int(i), 8] = pred[int(i), 8] + 1
        elif k == '9':
            pred[int(i), 9] = pred[int(i), 9] + 1

for i in range(10):
    TP = TP + pred[i,i]

acc = TP / s * 100
print("Accuracy: " + str(acc))
table = PrettyTable()
table.title = ('Predicted Values')
table.field_names = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
table.add_row(pred[0,:])
table.add_row(pred[1,:])
table.add_row(pred[2,:])
table.add_row(pred[3,:])
table.add_row(pred[4,:])
table.add_row(pred[5,:])
table.add_row(pred[6,:])
table.add_row(pred[7,:])
table.add_row(pred[8,:])
table.add_row(pred[9,:])
table.add_column("True Values", (0,1,2,3,4,5,6,7,8,9))

print(table)

## 3. Algebra

W plikach tekstowych `matrix_P.txt` oraz `matrix_Q.txt` znajduje się po 8 pomiarów 4-wymiarowej zmiennej (8 wierszy po 4 wartości).
* Wczytaj macierze do osobnych zmiennych `P` , `Q`.
* Korzystając z biblioteki `numpy` zaimplementuj funkcję obliczającą wzór:

$$
D_{KL}(P, Q) = \frac{1}{2} \mathrm{tr}\{(\boldsymbol{\Sigma}_p^{-1} + \boldsymbol{\Sigma}_q^{-1})(\boldsymbol{\mu}_p - \boldsymbol{\mu}_q)(\boldsymbol{\mu}_p - \boldsymbol{\mu}_q)^T + \boldsymbol{\Sigma}_p \boldsymbol{\Sigma}_q^{-1} + \boldsymbol{\Sigma}_q \boldsymbol{\Sigma}_p^{-1} - 2 \boldsymbol{I}\}
$$

gdzie:

$\mathrm{tr}$ – ślad macierzy (trace)

$\boldsymbol{\Sigma}$ – macierz kowariancji

$\boldsymbol{\mu}$ – uśredniony wektor ze wszystkich pomiarów

$\boldsymbol{I}$ – macierz jednostkowa

> Dla podanych danych funkcja powinna zwrócić wartość ok. 6.03

In [ ]:
import numpy as np
import statistics
from io import StringIO

with open("matrix_P.txt") as f:
    linesP = f.readlines()
with open("matrix_Q.txt") as f:
    linesQ = f.readlines()

P = np.zeros((8,4))
Q = np.zeros((8,4))
meanQ = np.zeros(4)
meanP = np.zeros(4)

for i in range(8):
    cP = StringIO(linesP[i])
    P[i,:] = np.loadtxt(cP)

    cQ = StringIO(linesQ[i])
    Q[i,:] = np.loadtxt(cQ)

for i in range(4):
    meanP[i] = statistics.mean(P[:,i])
    meanQ[i] = statistics.mean(Q[:,i])

covP = np.cov(np.transpose(P))
covQ = np.cov(np.transpose(Q))

D =0.5 * np.matrix.trace((np.linalg.inv(covP) + np.linalg.inv(covQ))*(meanP - meanQ) * np.transpose(meanP - meanQ) + covP * np.linalg.inv(covQ) + covQ * np.linalg.inv(covP) - 2 * np.eye(4))

print("D_KL(P,Q): " + str(D))